We repeat the greenscreen replacement from lesson 1 and lesson 5,
but this time with nearest neighbors algorithm.

Here we display the image.

In [ ]:
import numpy as np
from keras.preprocessing import image

img = image.load_img("imgs/greenML.png")

display(img)

Below we trim the edges of the image.

In [ ]:
arr = image.img_to_array(img)
# Trim off edges
arr = arr[:696,:]
display(image.array_to_img(arr,scale=False))

In [ ]:
# Given a rectangle of pixels, return an ndarray of all unique pixel values.
# Selecting unique pixels now noticeably reduce classification time later,
# e.g., from 68 seconds to 30 seconds.
def pixel_list(pixels, name):
    display(image.array_to_img(pixels, scale=False))
    pixels = np.reshape(pixels, (-1, 3))
    unique_pixels = set()
    for p in pixels:
        unique_pixels.add(tuple(p.tolist()))
    unique_pixels = np.array([list(p) for p in unique_pixels])
    print(name, len(pixels), '->', len(unique_pixels))
    return unique_pixels

In this example, we isolate out the bacground and convert it to a dataset of positive examples, `yesList`.

In [ ]:
# background
yesList = pixel_list(arr[:,:360], 'yesList')

Now we'll isolate out the foreground and make a dataset of foreground pixels called `noList`.

In [ ]:
# foreground
noList = pixel_list(arr[30:,547:620], 'noList')

We finalize our dataset here, with a variable `alldat` which cointains our list of pixels, and `labs` which is our list of labels for each pixel, `1` for green background pixels and `0` for foreground pixels

In [ ]:
# Build a list of pixels for both positive and negative examples.
alldat = np.concatenate((yesList,noList))

# labels
labs = np.concatenate((np.ones(len(yesList)), np.zeros(len(noList))))

Build and train a nearest neighbors classifier to separate the background from the foreground.
We will use k=1. Experiments showed that larger k values such as 3, 5, or 9 made little difference.

In [ ]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=1)
clf = clf.fit(alldat, labs)

Now we'll display the predictions of the classifier for each pixel in our image.
The resulting mask looks good overall, except for some glitches around the green card held by the person.

In [ ]:
# Turn the pixels in the image into a list
flat = np.reshape(arr,(-1,3))
out = clf.predict(flat)
# Reshape the output as a 2 dimensional list instead of 1 dimensional
out = np.reshape(out,(-1,1))
# Now, concatenate this list it itself three times to make something
#  like a color. Reshape the resulting list into the shape of the original
#  image.
newarr = np.reshape(np.concatenate((out, out, out),1),arr.shape)
# Display the image
display(image.array_to_img(newarr))

With this classifier, we can replace the background with a new image of a forest

We display the image of the forest, `img` below and covert the image into an array, `bkg`.

`composite` creates a new image based on a mask.  For each pixel in the new image:

- If the `mask` predicts the pixel to be in the background, we include the corresponding pixel from `background`.  
- If the `mask` predicts predicts the pixel to be in the foreground, we include the corresponding pixel from the `foreground`

In [ ]:
def composite(mask, foreground, background):
  ishift = 157
  for i in range(min(background.shape[0],foreground.shape[0]+ishift)):
    for j in range(min(background.shape[1], foreground.shape[1])):
      fgi = i - ishift
      if fgi >= 0 and not mask[fgi][j][0]: background[i][j] = foreground[fgi][j]
  display(image.array_to_img(background,scale=False))

Display the composite image. A green outline can be seen around the person,
similar to the results from lesson 1 and lesson 5.
The glitches around the green card is visible, though less obvious.

In [ ]:
img = image.load_img("imgs/forest.jpg")
bkg = image.img_to_array(img)
composite(newarr,arr,bkg)

Here is an attempt to remove the glitches around the green card, but it didn't turn out well.

In [ ]:
# alternative foreground that includes the green card
noList2 = pixel_list(arr[250:,620:740], 'noList2')

In [ ]:
alldat2 = np.concatenate((yesList, noList2))
labs2 = np.concatenate((np.ones(len(yesList)), np.zeros(len(noList2))))

clf2 = neighbors.KNeighborsClassifier(n_neighbors=1)
clf2 = clf2.fit(alldat2, labs2)

flat = np.reshape(arr,(-1,3))
out = clf2.predict(flat)
out = np.reshape(out,(-1,1))
newarr2 = np.reshape(np.concatenate((out, out, out),1),arr.shape)

Display the new mask. The glitches around the green card is now much less obvious, though still present.
However, we now see lots of glitches around the person, and at the upper-right corner of the image.

In [ ]:
display(image.array_to_img(newarr2))

In [ ]:
img = image.load_img("imgs/forest.jpg")
bkg = image.img_to_array(img)
composite(newarr2,arr,bkg)